In [2]:
import thinkbayes
import pandas as pd
import seaborn as sns

%pylab inline
pylab.rcParams['figure.figsize'] = (16, 8)

Populating the interactive namespace from numpy and matplotlib


# 예측
___
## 1. 보스턴 브루인스 문제

>- 2010-11 하키리그 결승전에서 보스턴 브루인스와 벤쿠버 캐넉스를 만남
>- 7전 4선승제에서 처음 두 경기를 0-1, 2-3으로 패배
>- 이후 두 경기를 8-1, 4-0으로 승리
>- 다음 경기를 이길 확률과, 결승전을 승리할 확률은 어떻게 되는가?

- 위 문제에 답하기 위해선 두가지 가정을 해야 함
    > 1. 경기 중 어떤 때든 골이 들어갈 확률은 동일하다고 보는 포아송 분포를 따름  
    > 2. 특정 상대에 대해 각 팀의 장기적 평균 골 수는 $\lambda$를 따름

- 위 문제에 답하기 위한 절차는 다음과 같음
> 1. $\lambda$에 대한 사전 분포를 고르기 위해 전 게임에 대한 통계를 확인  
> 2. 각 팀에 대한 $\lambda$ 추정을 위해 처음 네 게임의 점수를 사용  
> 3. $\lambda$의 사후 분포를 통해 각 팀 별 골의 분포 골 차이의 분포, 각 팀이 다음에 이길 확률을 계산  
> 4. 각 팀이 이번 시리즈에서 이길 확률을 계산

___
- 사전분포는 NHL 2010-11 시즌의 팀장 평균 골 수를 사용하여 근사
- 분포는 $N(2.8, 0.3)$의 가우시안 분포를 따름
- 가우시안 분포는 연속형이나, `thinkbayes.MakeGaussianPmf`를 통해 이산 Pmf로 추정
- 여기서 `mu`는 평균, `sigma`는 표준편차, `num_sigma`는 표준편차의 배수('2'sigma, '6'sigma), `n`은 이산 Pmf의 범위 갯수
```python
def MakeGaussianPmf(mu, sigma, num_sigmas, n = 101):
    pmf = Pmf()
    low = mu - num_sigmas * sigma
    high = mu + num_sigmas * sigma
    
    for x in np.linspace(low, high, n):
        p = scipy.stats.norm.pdf(mu, sigma, x)
        pmf.Set(x, p)
    pmf.Normalize()
    return pmf
```

- 하키 문제로 돌아가서 $\lambda*$에 대한 suite의 정의는 아래와 같이 내릴 수 있음
- 사전분포는 평균 2.7에 표준편차가 0.3인 가우시안 분포를 따르고, 4 sigma의 범위로 pmf를 구성

In [12]:
class Hockey(thinkbayes.Suite):
    
    def __init__(self):
        pmf = thinkbayes.MakeGaussianPmf(mu = 2.7, sigma = 0.3, num_sigmas= 4)
        thinkbayes.Suite.__init(self, pmf)

___
## 2. 포아송 프로세스